In [379]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.sparse import lil_matrix, csr_matrix, coo_matrix, hstack, save_npz, load_npz

from sklearn.linear_model import LogisticRegression

from tqdm import tqdm
import copy

import torch
import torch.nn as nn

from scipy.stats import spearmanr, kendalltau

# Выгрузка и обработка данных


In [197]:
# with open("./chgk/players.pkl", 'rb') as f:
#     players = pickle.load(f)
    
with open("./chgk/results.pkl", 'rb') as f:
    results = pickle.load(f)
    
with open("./chgk/tournaments.pkl", 'rb') as f:
    tournaments = pickle.load(f)

Возьмем в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.

Турниры, в которых есть данные о составах команд (количество команд > 1) и повопросных результатах

Заменили "?" на 0 в масках

In [198]:
results_2019 = defaultdict(list)
results_2020 = defaultdict(list)
for tour, teams in results.items():
    year = int(tournaments[tour]['dateStart'][:4])
    if (year == 2019 or year == 2020) and len(teams) > 1:
        for team in teams:
            if 'mask' in team and team['mask'] and team['teamMembers']:
                mask = team['mask'].replace('?', '0')
                if year == 2019:
                    results_2019[tour].append({
                        'team_id': team['team']['id'],
                        'mask': mask,
                        'members': [member['player']['id'] for member in team['teamMembers']]
                    })
                else:
                    results_2020[tour].append({
                        'team_id': team['team']['id'],
                        'position': team['position'],
                        'members': [member['player']['id'] for member in team['teamMembers']]
                    })

Почистим маски для results_2019. Выкинем команды которые ответили не на максимум вопросов. Выкинем вопросы где 'X'

In [307]:
X_train = []
question_count = 0
for tour, teams in results_2019.items():
    teams.sort(key=lambda x: len(x['mask']), reverse=True)
    masks = pd.DataFrame([list(team['mask']) for team in teams])
    masks.dropna(inplace=True)
    for col in masks:
        if not masks[masks[col] == 'X'].empty:
            masks.drop(columns=[col], inplace=True)
    masks = masks.astype(int).values
    
    question_numbers = np.arange(question_count, question_count + masks.shape[1])
    question_count = question_count + masks.shape[1]
    
    for i in range(masks.shape[0]):
        tmp = np.stack(np.broadcast_arrays(tour, 
                                           teams[i]['team_id'], 
                                           teams[i]['members'], 
                                           question_numbers.reshape(-1,1), 
                                           masks[i].reshape((-1,1))))
        t = tmp.T
        X_train.extend(t.reshape(len(teams[i]['members']) * len(question_numbers), 5).tolist())

In [309]:
np.random.shuffle(X_train)

In [317]:
X_train_pd = pd.DataFrame(X_train, 
                          columns=['tour', 
                                   'team_id', 
                                   'player',
                                   'question_id',
                                   'answer']).astype({'tour': 'int32', 
                                                      'team_id': 'int32',
                                                      'player': 'int32',
                                                      'question_id': 'int32',
                                                      'answer': 'int8'})
X_train_pd.to_csv('./train.csv', index=False)

In [19]:
train_data = pd.read_csv('./train.csv')

In [319]:
train_data.head()

,tour,team_id,player,question_id,answer
0,5401,2,13782,5274,1
1,5854,74349,46688,23945,0
2,5728,69316,190661,19361,0
3,5729,65981,73522,19389,1
4,5852,73088,79222,23885,0


### One-Hot encoding

In [323]:
from sklearn.preprocessing import OneHotEncoder

In [357]:
encoder = OneHotEncoder(dtype=int)

In [358]:
oh_players = encoder.fit_transform(train_data.player.values.reshape(-1,1))

In [359]:
encoder.categories_[0]

array([    15,     16,     23, ..., 224482, 224539, 224542])

In [360]:
encoder_q = OneHotEncoder(dtype=int)

In [361]:
oh_questions = encoder_q.fit_transform(train_data.question_id.values.reshape(-1,1))

In [363]:
encoder_q.categories_[0]

array([    0,     1,     2, ..., 33228, 33229, 33230])

In [365]:
oh_players.shape

(17753292, 57424)

In [367]:
oh = hstack([oh_players, oh_questions])

In [368]:
oh.shape

(17753292, 90655)

# Построим baseline-модель на основе логистической регрессии, которая будет обучать рейтинг-лист игроков.

В качестве обучающей выборки возьмем всевозможные пары игрок-вопрос. Обучив модель, получим вектор параметров, которые будут соответствовать "рейтингу" конкретного игрока и "сложности" конкретного вопроса.

In [369]:
baseline = LogisticRegression()

In [370]:
baseline.fit(oh, train_data.answer)

/Users/yuliya/miniconda3/envs/nn/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [371]:
baseline.coef_[0]

array([ 0.21897575,  1.08875896,  0.18195477, ...,  0.9309888 ,
       -2.08718528, -1.64767097])

# Результаты нового турнира с известными составами

Пусть рейтинг команды - средний рейтинг игроков. Отранжируем команды в каждом турнире по полученному рейтингу и вычислим коррелиции Спирмена и Кендалла с реальными положениями команд в турнирах.

In [442]:
def eval_rating(model_coef):
    rating_dict = dict(zip(encoder.categories_[0], model_coef[:len(encoder.categories_[0])]))
    total_spearman = 0
    total_kendalltau = 0
    for tour, teams in results_2020.items():
        teams_rating = [np.mean([rating_dict[member] if member in rating_dict else 0 for member in team['members']])
                       for team in teams]
        teams_pos = [team['position'] for team in teams]
        total_spearman += spearmanr(teams_pos, teams_rating)[0]
        total_kendalltau += kendalltau(teams_pos, teams_rating)[0]
    return abs(total_spearman / len(results_2020)), abs(total_kendalltau / len(results_2020))

In [444]:
sp, kend = eval_rating(baseline.coef_[0])

In [445]:
print(f'Корреляция Спирмена: {sp}')
print(f'Корреляция Кендалла: {kend}')

Корреляция Спирмена: 0.7410600750145123
Корреляция Кендалла: 0.5842877279564944


# ЧГК — это всё-таки командная игра.

Сделаем несколько предположений:
 
Ответ команды:
$$
z = 
\begin{cases}
1, &\text{если хотя бы один игрок из команды ответил верно} \\
0, &\text{если все игроки ответили неверно}
\end{cases}
$$

Будем моделировать вероятность того, что игрок правильно ответил на вопрос, с помощью логит функции:
$$
p(y = 1 | \mathbf{x}) = \sigma(\eta(\mathbf{x}))
$$
где $\mathbf{x}$ $-$ вектор игрок + вопрос

Используем ЕМ-алгоритм. В качестве скрытых переменных возьмем ответ игрока на вопрос.

E-шаг: 
$$
y_i^{(m+1)} = \mathbb{E}(y_i) = p(y_i = 1 | z) = 
\begin{cases}
\dfrac{\sigma\left(\eta^{(m)}(x_i)\right)}{1 - \prod\limits_{j}\left(1 - \sigma\left(\eta^{(m)}(x_j)\right)\right)}, &\text{z = 1} \\
0, &z = 0
\end{cases}
$$

M-шаг: обучаем лог регрессию при новых скрытых параметрах. Получим новые $\eta^{(m+1)}$.

В качестве начального приближения $\eta^{(0)}$ возьмем параметры baseline модели.

In [392]:
def to_sparse_tensor(coo_matrix):
    values = coo_matrix.data
    indices = np.vstack((coo_matrix.row, coo_matrix.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo_matrix.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [393]:
sparse_tensor = to_sparse_tensor(oh)

In [484]:
def train_model(
    model,
    opt,
    loss_function,
    X,  
    y,  
    iters = 10
):
    model.train()
    for i in tqdm(range(iters)):     
        logits = model(X).flatten()
        loss = loss_function(torch.sigmoid(logits), y)
        loss.backward()
        opt.step()
        opt.zero_grad()      
    return model

In [485]:
tmp = train_data[['team_id', 'question_id']].copy()
def e_step(model):
    model.eval()
    logits = model(sparse_tensor).detach()
    preds = torch.sigmoid(logits).flatten().numpy()
    tmp['vals'] = 1 - preds
    denominator = res[['team_id', 'question_id']].merge(
                        tmp.groupby(['team_id', 'question_id'], as_index=False).agg({'vals':'prod'}),
                        how='left',
                        on=['team_id', 'question_id'])['vals'].values 
    return train_data.answer * preds / (1 - denominator)

In [486]:
def m_step(model, opt, loss_func, y_m):
    model = train_model(model, opt, loss_func,  sparse_tensor, torch.tensor(y_m))
    return model

In [487]:
def em_algo(model, opt, loss_func,  y, iterations = 5):
    best_model = copy.deepcopy(model)
    best_kend = 0.5842877279564944
    spearman_list = []
    kendall_list = []
    for i in range(iterations):
        print(f'Итерация {i}')
        new_y = e_step(model)
        model = m_step(model, opt, loss_func,  torch.tensor(new_y, dtype = torch.float32))
        sp, kend = eval_rating(model.weight.data.flatten().numpy())
        spearman_list.append(sp)
        kendall_list.append(kend)
        if kend > best_kend:
            best_kend = kend
            best_model = copy.deepcopy(model)
        print(f'Корреляция Спирмена: {sp}')
        print(f'Корреляция Кендалла: {kend}')
        print()
    return best_model, spearman_list, kendall_list

In [490]:
#init model
model = nn.Linear(sparse_tensor.shape[1], 1)
with torch.no_grad():
    model.weight.data = torch.tensor(baseline.coef_, dtype=torch.float32)

opt = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_function = nn.BCELoss()

In [491]:
best_model, spearman_list, kendall_list = em_algo(model, opt, loss_function, train_data.answer)

Итерация 0


/Users/yuliya/miniconda3/envs/nn/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


Корреляция Спирмена: 0.7409234456364032
Корреляция Кендалла: 0.5841089201466653

Итерация 1


100%|██████████| 10/10 [00:20<00:00,  2.01s/it]


Корреляция Спирмена: 0.7408568527897602
Корреляция Кендалла: 0.5840402301006333

Итерация 2


...

Корреляция Спирмена: 0.7407108702281988
Корреляция Кендалла: 0.5838747517919411

Итерация 3


100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


Корреляция Спирмена: 0.740659638962295
Корреляция Кендалла: 0.5837859469837579

Итерация 4


100%|██████████| 10/10 [00:20<00:00,  2.03s/it]


Корреляция Спирмена: 0.7405730752026731
Корреляция Кендалла: 0.583633732651929



Метрики уменьшаются, пока не удалось выяснить почему.

# “Рейтинг-лист” турниров по сложности вопросов

In [500]:
question_complex = baseline.coef_[0][len(encoder_q.categories_[0]):]

In [501]:
tour_rating_list = defaultdict(list)
for tour, question in np.unique(train_data[['tour', 'question_id']].values, axis=0):
    tour_rating_list[tour].append(question_complex[question])

In [502]:
tour_rating = {}
for tour, val in tour_rating_list.items():
    tour_rating[tour] = np.mean(val)

In [503]:
tour_rating

{4772: -0.30998198764148055,
 4973: -0.324412444573232,
 4974: -0.5433368808218821,
 4975: -0.36644568478632045,
 4986: -0.2488760893378258,
 5000: -0.334892107025736,
 5008: -0.10883938667509542,
 5009: -0.5151727967134823,
 5010: -0.5286644935939562,
 5011: -0.22736564793321343,
 5012: -0.06754638418747855,
 5013: -0.25331503991252124,
 5021: -0.3034117106803156,
 5025: -0.16906875419306688,
 5042: -0.48497580130091905,
 5052: -0.5706154728567275,
 5053: -0.1590047473737678,
 5055: -0.1827479498147011,
 5056: -0.0057238080259013665,
 5060: -0.2412970160633798,
 5061: -0.2908284426446859,
 5065: -0.11368514584019793,
 5070: -0.09871035847355873,
 5071: 0.0071577223441462124,
 5074: -0.23324855663103858,
 5078: 0.03830235795680511,
 5083: -0.16184505141428449,
 5097: -0.30514609831443024,
 5098: -0.16534509322410706,
 5103: 0.13818557162001655,
 5108: 0.013370790014258015,
 5109: -0.38312066594950694,
 5110: -0.22972675691919023,
 5111: -0.12284848798611746,
 5112: -0.34450433480883447

In [504]:
sorted_tours = sorted(tour_rating.items(), key=lambda x: x[1], reverse=True)

In [505]:
#lest popular
for tour, rating in sorted_tours[:20]:
    print(tournaments[tour]['name'])

Гран-при Славянки. 4 этап
Синхрон Беловежской Зимы. День второй.
Молодёжный Чемпионат Армении
Донат
Кубок Тель-Авива
Dichtenszeit-2
Беловежская Зима
Синхрон ОК СПбГУ
Игра в бисер
Белодачники пишут
Gamer - 1
ОЧВР. 1 тур
Joystick Cup
Чемпионат Мира. Этап 2 Группа С
Синхрон Беловежской Зимы. День первый.
Из Минска с любовью. Этап 1.
Кубок Новополоцка
Чемпионат Мира. Этап 3. Группа С
Трэцяя актава. Ліга нацый: Беларусь
Синхрон открытого чемпионата Чехии


In [506]:
#most popular
for tour, rating in sorted_tours[-20:]:
    print(tournaments[tour]['name'])

Синхрон Майка Иванова
Чемпионат Азербайджана
Межфакультетский кубок МГУ
Школьная лига. III тур.
Пляжное ЧГК
Gamer - 2
Студенческая лига ЧТ
Золотые огни: часть 2
Турнир малых факультетов МГУ
Гран-при Славянки. 6 этап
Нескучный кубок
Зеркало Я.Сова
Осенняя поляна
Угрюмый Ёрш
Лёгкий Смоленск
Дилижанские игры
Всеармянский Интеллектуальный Фестиваль
Голова профессора Доуэля
Borisov Brain Cup
Славянка без раздаток. 4 этап
